<a href="https://colab.research.google.com/github/Carolinsrainbow/UC_SistemasRecomendadores/blob/main/Tutorial_ALS_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial ALS con Implicit I

MAN 3160 - Sistemas Recomendadores

En este tutorial vamos a utilizar la biblioteca de Python [implicit](https://implicit.readthedocs.io/en/latest/quickstart.html) para implementar un sistema recomendador ALS.


## Importar Librerías

In [ ]:
# Instalamos librerías para descarcar y descomprimir archivos.

!pip install wget
!pip install zipfile36

In [ ]:
!pip3 install implicit --upgrade

In [ ]:
import pandas as pd
import zipfile
import numpy as np
import implicit
import scipy.sparse as sparse

C:\Users\alfi9\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Descarga del dataset

Al igual que en la tarea 1, para este tutorial utilizaremos el dataset MovieLens-100k proporcionado de [la Universidad de Minnesota](https://grouplens.org/).

In [ ]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [ ]:
with zipfile.ZipFile("ml-100k.zip", 'r') as zip_ref:
    zip_ref.extractall(".")

In [ ]:
dir_train = 'ml-100k'

# Generamos los títulos de las columnas del archivo items.

columns = ['movieid', 'title', 'release_date', 'video_release_date', \
           'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', \
           'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', \
           'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', \
           'Thriller', 'War', 'Western']

In [ ]:
# Primero creamos el dataframe con los datos
df_train = pd.read_csv(f'{dir_train}/u2.base',
                         sep='\t',
                         names=['userid', 'itemid', 'rating', 'timestamp'],
                         header=None)

In [ ]:
df_train.head()

,userid,itemid,rating,timestamp
0,1,3,4,878542960
1,1,4,3,876893119
2,1,5,3,889751712
3,1,6,5,887431973
4,1,7,4,875071561


### Transfromación de dataset a feedback implícito

El dataset MovieLens cuenta con ratings dados por los usuarios a cada película. Esto significa que el dataset es explícito, por lo que un método como ALS no sería el apropiado en este caso. Sin embargo, podemos procesar el set de datos para simular un dataset con feedback implícito a partir de MovieLens.

In [ ]:
# rating >= 3 , relevante (1) y rating menor a 3 es no relevante (0)
df_train.rating = [1 if x >=3 else 0 for x in df_train.rating ]

In [ ]:
df_train.head(20)

,userid,itemid,rating,timestamp
0,1,3,1,878542960
1,1,4,1,876893119
2,1,5,1,889751712
3,1,6,1,887431973
4,1,7,1,875071561
5,1,10,1,875693118
6,1,11,0,875072262
7,1,12,1,878542960
8,1,13,1,875071805
9,1,14,1,874965706


In [ ]:
# Cargamos el dataset con los items
df_items = pd.read_csv(f'{dir_train}/u.item',
                        sep='|',
                        index_col=0,
                        names = columns,
                        header=None,
                        encoding='latin-1')

In [ ]:
df_items.head()

,title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieid,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


### Preprocesamiento de los datos a formato sparse

In [ ]:
user_items = {}
itemset = set()

for row in df_train.itertuples():
    if row[3]:
        if row[1] not in user_items:
            user_items[row[1]] = []

        user_items[row[1]].append(row[2])
        itemset.add(row[2])

itemset = np.sort(list(itemset))

sparse_matrix = np.zeros((len(user_items), len(itemset)))

for i, items in enumerate(user_items.values()):
    sparse_matrix[i] = np.isin(itemset, items, assume_unique=True).astype(int)

user_item_matrix = sparse.csr_matrix(sparse_matrix)

user_ids = {key: i for i, key in enumerate(user_items.keys())}
items_ids = {key: i for i, key in enumerate(itemset)}

In [ ]:
sparse_matrix.shape

(943, 1550)

## ALS (Alternating Least Squares)

### Entrenamiento de método

In [ ]:
# Definimos y entrenamos el modelo con optimización ALS
model_als = implicit.als.AlternatingLeastSquares()
model_als.fit(user_item_matrix)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:01<00:00,  9.54it/s]


In [ ]:
user_item_matrix

<943x1550 sparse matrix of type '<class 'numpy.float64'>'
	with 65962 stored elements in Compressed Sparse Row format>

### Generar recomendaciones

In [ ]:
def show_recommendations(model, user, n):
    recommendations = model.recommend(userid=user_ids[user], user_items=user_item_matrix[user_ids[user]], N=n)
    return df_items.loc[itemset[recommendations[0]]]['title']

Ejemplo de recomendación y búsqueda de items similares con los factores latentes ya entrenados:

In [ ]:
show_recommendations(model_als, user=61, n=10)

movieid
313                  Titanic (1997)
307    Devil's Advocate, The (1997)
245         Devil's Own, The (1997)
302        L.A. Confidential (1997)
272        Good Will Hunting (1997)
315                Apt Pupil (1998)
322           Murder at 1600 (1997)
300            Air Force One (1997)
271        Starship Troopers (1997)
316       As Good As It Gets (1997)
Name: title, dtype: object

### Ver títulos similares.

In [ ]:
def show_similar_movies(model, item, n=10):
    sim_items = model.similar_items(items_ids[item], n)[0]
    sim_items = [itemset[i] for i in sim_items]
    return df_items.loc[sim_items]['title']

In [ ]:
show_similar_movies(model_als, 313, 10)

movieid
313                       Titanic (1997)
1293                     Star Kid (1997)
272             Good Will Hunting (1997)
751           Tomorrow Never Dies (1997)
315                     Apt Pupil (1998)
1612             Leading Man, The (1996)
354           Wedding Singer, The (1998)
895                      Scream 2 (1997)
1236    Other Voices, Other Rooms (1997)
1294    Ayn Rand: A Sense of Life (1997)
Name: title, dtype: object